In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import random
import seaborn as sns

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

from keras.models import Sequential
from keras.utils import np_utils
from keras import optimizers
from keras import backend as K
from keras.layers import Dense,Activation,Flatten,MaxPool2D,Dropout,Conv2D,BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator



In [ ]:
size=150
data=[]
label_dict={"NORMAL":0,"PNEUMONIA":1}

def load_data(directory):
    for sub_directory in os.listdir(directory):
        if sub_directory == "NORMAL":
            inner_directory=os.path.join(directory,sub_directory)
            for i in os.listdir(inner_directory):
                img=cv2.imread(os.path.join(inner_directory,i),0)
               # img=cv2.equalizeHist(img)
                img=cv2.resize(img,(size,size))
                #img=cv2.threshold(img,)
                data.append([img,label_dict[sub_directory]])
                
        if sub_directory == "PNEUMONIA":
            inner_directory=os.path.join(directory,sub_directory)
            for i in os.listdir(inner_directory):
                img=cv2.imread(os.path.join(inner_directory,i),0)
                img=cv2.resize(img,(size,size))
                data.append([img,label_dict[sub_directory]])
                
    random.shuffle(data)  
    return np.array(data)
                
                


In [ ]:
train=load_data('../input/chest-xray-pneumonia/chest_xray/train')
val = load_data('../input/chest-xray-pneumonia/chest_xray/val')
test = load_data('../input/chest-xray-pneumonia/chest_xray/test')

In [ ]:
val

In [ ]:
total_count=0
pneumonia1=0
normal1=0
i=0
normal=[]
pneumonia=[]

for i in val:
    total_count+=1
    if i[1]==0:
        normal1+=1
        normal.append(i[0])
    else:
        pneumonia1+=1
        pneumonia.append(i[0])
        
print('Total_count is '+str(total_count)) 
print('Normal '+str(normal1))
print('Pneumonia '+str(pneumonia1) )

            

In [ ]:
r=2
c=4
fig=plt.figure(figsize=(18,18))
fig,ax=plt.subplots(2,4)
for i in range(r):
    for j in range(c):
        if j<1:
            ax[i,j].imshow(normal[j])
            ax[i,j].set_title('Normal')
        else:
            ax[i,j].imshow(pneumonia[j])
            ax[i,j].set_title('Pneumonia')

        

In [ ]:
plt.imshow(pneumonia[1])
image=pneumonia[1]
#image= cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
#image= cv2.threshold(image, 120, 255, cv2.THRESH_BINARY) 
image=cv2.equalizeHist(image)
#ret,image= cv2.threshold(image, 120, 255, cv2.THRESH_BINARY)
image = cv2.adaptiveThreshold(image,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

plt.imshow(image)

In [ ]:
x_train=[]
y_train=[]

for x,y in train:
    x_train.append(x)
    y_train.append(y)

In [ ]:
x_test=[]
y_test=[]

for x,y in train:
    x_test.append(x)
    y_test.append(y)

In [ ]:
x_val=[]
y_val=[]

for x,y in train:
    x_val.append(x)
    y_val.append(y)

In [ ]:
y_test[1]

In [ ]:
x_train=np.array(x_train)/255
x_test=np.array(x_test)/255
x_val=np.array(x_val)/255

In [ ]:
x_train=x_train.reshape(-1,size,size,1)
y_train=np.array(y_train)

In [ ]:
x_train

In [ ]:
x_test=x_train.reshape(-1,size,size,1)
y_test=np.array(y_test)
x_val=x_train.reshape(-1,size,size,1)
y_val=np.array(y_val)

In [ ]:
image_gen=ImageDataGenerator(rotation_range=30,width_shift_range=0.2,
                            height_shift_range=0.2,
                            featurewise_center=False,
                            samplewise_center=False,
                            featurewise_std_normalization=False,
                            samplewise_std_normalization=False,
                            zca_whitening=False,
                            zoom_range=0.2,
                            horizontal_flip=True)

In [ ]:
model=Sequential()
model.add(Conv2D(64,(3,3),strides=1,padding='same',activation='relu',input_shape=(150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D ((2,2),strides =2,padding='same'))

model.add(Conv2D(64,(3,3),strides=1,padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2,padding='same'))

model.add(Conv2D(128,(3,3),strides=1,padding='same',activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2,padding='same'))

model.add(Conv2D(256,(3,3),strides=1,padding='same',activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2),strides=2,padding='same'))

model.add(Flatten())

model.add(Dense(units=128,activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(units=1,activation='sigmoid'))

model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

model.summary()


In [ ]:
reduce_lr=ReduceLROnPlateau(monitor='val_accuracy',patience=2,verbose=1,factor=0.25,min_lr=0.00001)

In [ ]:
history=model.fit(image_gen.flow(x_train,y_train,batch_size=25),epochs=12,
                 validation_data=(image_gen.flow(x_val,y_val)),callbacks=[reduce_lr])

In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','val'],loc='upper left')
plt.show()

In [ ]:
model.evaluate(x_test,y_test)

In [ ]:
prediction=model.predict_classes(x_test)
print(classification_report(prediction,y_test))

In [ ]:
confusion_matrix(prediction,y_test)

In [ ]:

from mlxtend.plotting import plot_confusion_matrix
#from sklearn.metrics import confusion_matrix
mat=confusion_matrix(prediction,y_test)
plot_confusion_matrix(mat,figsize=(9,9),colorbar=True,)